In [2]:
from fol import evaluate
from mlx_lm import load, generate 
from datasets import load_dataset
from folio_dataset import create_prompt, instruction, get_example_prompt_str, fol_to_nltk, convert_to_nltk_rep

folio_train = load_dataset('yale-nlp/FOLIO', split='train')
folio_val = load_dataset('yale-nlp/FOLIO', split='validation')

folio_train = folio_train.map(fol_to_nltk)
folio_val = folio_val.map(fol_to_nltk)

example_prompt_str = get_example_prompt_str(folio_train, n_shots=4)

folio_val = folio_val.map(
    create_prompt,
    fn_kwargs=dict(
        instruction=instruction,
        example_prompt_str=example_prompt_str
    )
)

/Users/zhijiezhu/.pyenv/versions/3.11.8/envs/mlx/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.
Map: 100%|██████████| 203/203 [00:00<00:00, 13050.35 examples/s]


In [2]:
print(folio_val[0]['prompt'])

The following is a first-order logic (FOL) problem.
The problem is to determine whether the conclusion follows from the premises.
The premises are given in the form of a set of first-order logic sentences.
The conclusion is given in the form of a single first-order logic sentence.
The task is to translate each of the premises and conclusions into FOL expressions,so that the expressions can be evaluated by a theorem solver to determine whether the conclusion follows from the premises.
Expressions should be adhere to the format of the provided examples.

<PREMISES>
All people who regularly drink coffee are dependent on caffeine.
People regularly drink coffee, or they don't want to be addicted to caffeine, or both.
No one who doesn't want to be addicted to caffeine is unaware that caffeine is a drug.
Rina is either a student who is unaware that caffeine is a drug, or she is not a student and is she aware that caffeine is a drug.
Rina  is either a student who is dependent on caffeine, or s

In [5]:
model, tokenizer = load("mlx-community/quantized-gemma-7b-it")
response = generate(model, tokenizer, prompt=folio_val[1]['prompt'], verbose=True, max_tokens=512)

Fetching 8 files: 100%|██████████| 8/8 [00:00<00:00, 69042.04it/s]


Prompt: The following is a first-order logic (FOL) problem.
The problem is to determine whether the conclusion follows from the premises.
The premises are given in the form of a set of first-order logic sentences.
The conclusion is given in the form of a single first-order logic sentence.
The task is to translate each of the premises and conclusions into FOL expressions,so that the expressions can be evaluated by a theorem solver to determine whether the conclusion follows from the premises.
Expressions should be adhere to the format of the provided examples.

<PREMISES>
All people who regularly drink coffee are dependent on caffeine.
People regularly drink coffee, or they don't want to be addicted to caffeine, or both.
No one who doesn't want to be addicted to caffeine is unaware that caffeine is a drug.
Rina is either a student who is unaware that caffeine is a drug, or she is not a student and is she aware that caffeine is a drug.
Rina  is either a student who is dependent on caffei

In [7]:
llm_translated_fols = []
for line in response.splitlines():
    if not line.startswith('FOL'):
        continue
    llm_translated_fols.append(line[5:])

evaluate(llm_translated_fols[:-1], llm_translated_fols[-1])

all x. (PerformInTalentShow(x) & MemberOfClub(x) -> AttendAndEngageWithSchoolEvents(x))
all x. (MemberOfClub(x) -> ((PerformInTalentShow(x) | -InactiveAndDisinterestedCommunityMember(x)))
all x. (ChaperoneHighSchoolDance(x) & MemberOfClub(x) -> -Student(x))
all x. (-InactiveAndDisinterestedCommunityMember(x) & MemberOfClub(x) -> ChaperoneHighSchoolDance(x))
all x. (YoungChildOrTeenager(x) & WishFurtherAcademicCareer(x) & MemberOfClub(x) -> Student(x))
MemberOfClub(Bonnie) & ((AttendAndEngageWithSchoolEvents(Bonnie) & Student(Bonnie)) | (-AttendAndEngageWithSchoolEvents(Bonnie) & -Student(Bonnie)))
(YoungChildOrTeenager(Bonnie) & WishFurtherAcademicCareer(Bonnie) & ChaperoneHighSchoolDance(Bonnie)) | (-YoungChildOrTeenager(Bonnie) & WishFurtherAcademicCareer(Bonnie) & -ChaperoneHighSchoolDance(Bonnie)) -> Student(Bonnie) | -InactiveAndDisinterestedCommunityMember


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


'Uncertain'

In [9]:
folio_val[1]['label']

'True'

In [4]:
for example in folio_val:
    print(type(example), example)

<class 'dict'> {'story_id': 380, 'premises': 'People in this club who perform in school talent shows often attend and are very engaged with school events.\nPeople in this club either perform in school talent shows often or are inactive and disinterested community members.\nPeople in this club who chaperone high school dances are not students who attend the school.\nAll people in this club who are inactive and disinterested members of their community chaperone high school dances.\nAll young children and teenagers in this club who wish to further their academic careers and educational opportunities are students who attend the school. \nBonnie is in this club and she either both attends and is very engaged with school events and is a student who attends the school or is not someone who both attends and is very engaged with school events and is not a student who attends the school.', 'premises-FOL': '∀x (InThisClub(x) ∧ PerformOftenIn(x, schoolTalentShow) → Attend(x, schoolEvent) ∧ VeryEng